In [ ]:
import os, sys, csv, gzip
import numpy as np
import pandas as pd
import scipy.sparse as sp
from scipy import io

import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams['pdf.fonttype'] = 42 ## Output Type 3 (Type3) or Type 42 (TrueType)
rcParams['font.sans-serif'] = 'Arial'

%matplotlib inline

from plots import COLORS10, enlarge_tick_fontsize
import seaborn as sns
sns.set_style('whitegrid')

from sklearn.cross_validation import (StratifiedKFold, cross_val_score)
from sklearn.metrics import *
from sklearn.decomposition import TruncatedSVD
from sklearn import ensemble
from sklearn.pipeline import Pipeline
import xgboost as xgb

from IPython import display

### Create Gene Set Libraries

In [ ]:
can_acro = pd.read_csv('Table_of_Cancer_Acro.csv', sep='\t', index_col=0)
can_acro = can_acro.index.tolist()

In [ ]:
meta = pd.read_csv('Piccolo/TCGA_Metadata.csv', sep=',',index_col='a_CGHubAnalysisID')
meta = meta['a_AliquotBarcode']

lst = []
for a in meta:
    lst.append(a.split('-')[2])
               
meta = pd.Series(data = lst, index=meta.index)

In [ ]:
meta2 = pd.read_csv('TCGAMETA.csv', sep=',')
for cancer in can_acro:
    #print cancer
    df = meta2[meta2['Study'] == cancer.split('[')[1].split(']')[0]]
    df.to_csv('TCGA_Meta_data/metadata_'+cancer, sep='\t')

In [ ]:
i = 0
count = 0
for fn in os.listdir('Piccolo'):
    if 'TCGA' in fn and 'Meta' not in fn and 'MAP': #not in fn and count < 10:
    #if fn == 'TCGA_ACC_tpm.tsv' or fn == 'TCGA_BLCA_tpm.tsv':# or fn == 'TCGA_CESC_tpm.tsv' or fn == 'TCGA_CHOL_tpm.tsv':
        count +=1

for fn in os.listdir('Piccolo'): 

    if 'TCGA' in fn and 'Meta' not in fn and 'MAP': #not in fn and i <=9:
    #if fn == 'TCGA_ACC_tpm.tsv' or fn == 'TCGA_BLCA_tpm.tsv':# or fn == 'TCGA_CESC_tpm.tsv' or fn == 'TCGA_CHOL_tpm.tsv':


        print '\r' + str(i+1) + " Out of " + str(count) + ' Files',
        
        example = pd.read_csv('Piccolo/'+fn, sep='\t', index_col=0)

        name = fn.split('_')[1]
        for n in can_acro:
            if name in n:
                name = n

        example.index.name = 'Gene'
        example.columns.name = 'Patients'
        example.name = name

        lst1 =[]
        lst2 = []
        for a in example.index:
            lst1.append(a.split('|')[5])
            lst2.append(a.split('|')[4])

        example.index = lst1
        example['Transcript'] = lst2
        example.insert(0,'Transcript', example.pop('Transcript'))

#         q = example.index.intersection(gene_to_plot)
#         example = example.T
#         example = example[q]
#         example = example.T

        example.drop_duplicates(inplace=True)

        example.reset_index(inplace=True)
        example.set_index(['index', 'Transcript'], inplace=True)
        example.name = name
        
        lst = []
        k = 0 
        for sample in example.columns:
            for ID in meta.index:
                k += 1
                if sample == ID:
                    lst.append(meta.ix[ID])
                print '\r'+ str(k) + " Out of " + str(len(example.columns)*len(meta.index)) + ' IDs',
        
        example.columns = lst
        example.sort_index(axis=1, ascending=False, inplace=True)
        print '\r'+ 'a',
        d = example.columns.duplicated()
        for col in example.columns[d].tolist():
            print '\r' + col,
            temp = example[col].T.mean()
            example.pop(col)
            example[col] = temp
        print '\r'+ 'b',
        for index in example.ix['TP53'].index:
            if index != 'TP53-021':
                example.ix['TP53', index] = np.nan 
        example.ix['LCE1C', 'LCE1C-002'] = np.nan
        print '\r'+ 'c',
        example.reset_index(inplace=True)
        print '\r'+ 'd',
        example.set_index('index', inplace=True)
        print '\r'+ 'e',
        example.pop('Transcript')
        print '\r'+ 'f',
        example.dropna(inplace=True)
        print '\r'+ 'g',
        example = example.T
        print '\r'+ 'h',
        a = ~example.columns.duplicated()
        a.tolist()
        example = example.T
        example = example[a]
        example = example.T
        print '\r'+ 'i',
        example.replace(0, np.nan, inplace=True)
        example.dropna(axis= 1, how='all', inplace=True)
        print '\r'+ 'j',
        example = example.apply(lambda x: (x-x.mean())/x.std(ddof=0), axis=1)
        print '\r'+ 'k',
        

        if os.path.exists('TCGA_Meta_data/metadata_'+name):
            m = pd.read_csv('TCGA_Meta_data/metadata_'+name, sep='\t')
            lst = []
            for label in m['ParticipantBarcode']:
                lst.append(label.split('-')[2])
            m.insert(0, 'ID',lst,)
            m.set_index('ID', inplace=True)

            up_df = pd.DataFrame()
            dn_df = pd.DataFrame()


            v = 0
            #print '\n'
            for patiant in example.index:

                print '\r'+ str(v+1) + " Out of " + str(len(example.index)) + ' Patients',

                uplst = []
                uplst = example.ix[patiant][example.ix[patiant] > 0].index.tolist()
                uplst.insert(0, patiant)
                updf = pd.DataFrame(data = uplst)
                updf = updf.T
                updf.set_index(0, inplace=True)
                if patiant in m.index:
                    updf.insert(0,'Tissue Site', m.ix[patiant].tumor_tissue_site)
                    updf.insert(0,'Race', m.ix[patiant].race)
                    updf.insert(0,'Age', m.ix[patiant].age_at_initial_pathologic_diagnosis)
                    updf.insert(0,'Gender', m.ix[patiant].gender)
                else:
                    updf.insert(0,'Tissue Site', np.nan)
                    updf.insert(0,'Race', np.nan)
                    updf.insert(0,'Age', np.nan)
                    updf.insert(0,'Gender', np.nan)

                dnlst = []
                dnlst = example.ix[patiant][example.ix[patiant] < 0].index.tolist()
                dnlst.insert(0, patiant)
                dndf = pd.DataFrame(data = dnlst)
                dndf = dndf.T
                dndf.set_index(0, inplace=True)
                if patiant in m.index:
                    dndf.insert(0,'Tissue Site', m.ix[patiant].tumor_tissue_site)
                    dndf.insert(0,'Race', m.ix[patiant].race)
                    dndf.insert(0,'Age', m.ix[patiant].age_at_initial_pathologic_diagnosis)
                    dndf.insert(0,'Gender', m.ix[patiant].gender)
                else:
                    dndf.insert(0,'Tissue Site', np.nan)
                    dndf.insert(0,'Race', np.nan)
                    dndf.insert(0,'Age', np.nan)
                    dndf.insert(0,'Gender', np.nan)

                up_df = pd.concat([up_df, updf])
                dn_df = pd.concat([dn_df, dndf])
                v += 1


            up_df.insert(0,'Tissue Site', up_df.pop('Tissue Site'))
            up_df.insert(0,'Race', up_df.pop('Race'))
            up_df.insert(0,'Age', up_df.pop('Age'))
            up_df.insert(0,'Gender',up_df.pop('Gender'))

            dn_df.insert(0,'Tissue Site', dn_df.pop('Tissue Site'))
            dn_df.insert(0,'Race', dn_df.pop('Race'))
            dn_df.insert(0,'Age', dn_df.pop('Age'))
            dn_df.insert(0,'Gender',dn_df.pop('Gender'))
            print '\r'+ 'p',
            up_df.to_csv('Output/GeneSetLibraries/'+name+'_up.gct', sep='\t')
            dn_df.to_csv('Output/GeneSetLibraries/'+name+'_dn.gct', sep='\t')


            i += 1